# Unit conversions

This notebook demonstrates some examples of different kinds of units, and the circumstances under which they are converted and displayed.

In [ ]:
# IMPORTS
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
sys.path.append('..')
import atomica as at
import matplotlib.pyplot as plt
import numpy as np
import sciris as sc
from IPython.display import display, HTML

CSS = """
.output {
    flex-flow: row wrap;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [ ]:
testdir = at.parent_dir()
P = at.Project(framework='unit_demo_framework.xlsx',databook='unit_demo_databook.xlsx')
P.load_progbook('unit_demo_progbook.xlsx')
res = P.run_sim('default','default',at.ProgramInstructions(start_year=2018))

This test example has the follow features

##### Parameters

- `recrate` - Duration in months
- `infdeath` - Weekly probability
- `susdeath` - Daily probability
- `foi` - Annual probability

In [ ]:
d = at.PlotData(res,outputs=['recrate','infdeath','susdeath','foi','sus:inf','susdeath:flow','dead'],pops='adults')
at.plot_series(d,axis='pops');

Notice that parameters are plotted in their native units. For example, a probability per day is shown as probability per day, matching the numbers that were entered in the databook.

Aggregating these units without specifying the aggregation method will result in either integration or averaging as most appropriate for the units of the underlying quantity:

In [ ]:
for output in ['recrate','infdeath','susdeath','foi','sus:inf','susdeath:flow','dead']:
    d = at.PlotData(res,outputs=output,pops='adults',t_bins=10)
    at.plot_bars(d);

##### Programs

 - `Risk avoidance` - Continuous
 - `Harm reduction 1` - Continuous
 - `Harm reduction 2` - Continuous
 - `Treatment 1` - One-off
 - `Treatment 2` - One-off
 
 Programs with continuous coverage cover a certain number of people every year:

In [ ]:
d = at.PlotData.programs(res,outputs='Risk avoidance',quantity='coverage_number')
at.plot_series(d);

Programs with one-off coverage cover a number of people at each time step. This is the number that gets returned by `Result.get_coverage()` but it is automatically annualized for plotting:

In [ ]:
annual_coverage = res.model.progset.programs['Treatment 1'].spend_data.vals[0]/res.model.progset.programs['Treatment 1'].unit_cost.vals[0]
timestep_coverage = res.get_coverage('number')['Treatment 1'][0]
print('Annual coverage = %g, Timestep coverage = %g' % (annual_coverage, timestep_coverage))

In [ ]:
d = at.PlotData.programs(res,outputs='Treatment 1',quantity='coverage_number')
at.plot_series(d)

These units are handled automatically when aggregating. For example, consider computing the number of people covered over a period of time:

In [ ]:
d = at.PlotData.programs(res,outputs='Treatment 1',quantity='coverage_number',t_bins=[2000,2000.5])
at.plot_bars(d);

d = at.PlotData.programs(res,outputs='Treatment 1',quantity='coverage_number',t_bins=[2000,2002])
at.plot_bars(d);

In [ ]:
d = at.PlotData.programs(res,outputs='Treatment 1',quantity='coverage_denominator',t_bins=[2000,2000.5])
at.plot_bars(d);

d = at.PlotData.programs(res,outputs='Treatment 1',quantity='coverage_number',t_bins=[2000,2002])
at.plot_bars(d);